In [16]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

#here, we loaded the mnist dataset as a one_hot encoding (only one element in array 1, others 0)


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
n_classes = 10        #10 classes, one for each digit
batch_size = 128      #size of betch that will be given at a time to our model

In [0]:
x = tf.placeholder('float', [None, 784])    #input 28x28 image converted to an array of 784 pixels(numbers)
y = tf.placeholder('float')                 #output

In [0]:
def conv2d(x, W):                                                               #2 dimentional convolution, W==weights
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def maxpool2d(x):                                                               #for spatial invariance, we do pooling, here we're doing max-pooling
    #                        size of window         movement of window
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [0]:
def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,1,32])),             #1st hidden layer, 5x5 convulation, 1 input, 32 output/features.
               'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),            #initially we are making all values random
               'W_fc':tf.Variable(tf.random_normal([7*7*64,1024])),             #fully connected layer on compressed image (7*7)
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}          

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),                    #biases are useful when all weights are zero.
               'b_conv2':tf.Variable(tf.random_normal([64])),                   
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    x = tf.reshape(x, shape=[-1, 28, 28, 1])                                    #reshaping an array of pixels to a flat image

    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])       #rectified linear function for first conv layer
    conv1 = maxpool2d(conv1)
    
    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])   
    conv2 = maxpool2d(conv2)

    fc = tf.reshape(conv2,[-1, 7*7*64])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output

In [0]:

def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    
    optimizer = tf.train.AdamOptimizer().minimize(cost)                         #here we are using Adam optimizer to reduce the cost
    
    hm_epochs = 10                                                              #epoch represents the no. of complete cycles...feed forward + backprop
    with tf.Session() as sess:                                                  #this is where our model actually starts
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
        
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))                     #calculates accuracy by finding the mean over correct values
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))



In [26]:
train_neural_network(x)

Epoch 0 completed out of 10 loss: 2183127.00579834
Epoch 1 completed out of 10 loss: 336011.6135864258
Epoch 2 completed out of 10 loss: 183348.78244781494
Epoch 3 completed out of 10 loss: 121244.92943763733
Epoch 4 completed out of 10 loss: 81452.38103103638
Epoch 5 completed out of 10 loss: 59420.420053482056
Epoch 6 completed out of 10 loss: 42443.262062072754
Epoch 7 completed out of 10 loss: 33823.78460842371
Epoch 8 completed out of 10 loss: 26247.725021582097
Epoch 9 completed out of 10 loss: 23826.8229967961
Accuracy: 0.9734
